## Marketing Campain

load environment variables

In [1]:
%load_ext dotenv
%dotenv ../../.env

In [22]:
import logging

from autogen_core import EVENT_LOGGER_NAME

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

In [2]:
from autogen import ConversableAgent, UserProxyAgent
import os

llm_config = {
    "config_list": [
        {
            "model": "gpt-4.1-nano", # 0
            "api_key": os.environ["OPENAI_API_KEY"],
        }
        , {
            "model": "gpt-4.1-mini", # 1
            "api_key": os.environ["OPENAI_API_KEY"],
        }
        , {
            "model": "gpt-4o-mini",  # 2
            "api_key": os.environ["OPENAI_API_KEY"],
        }
        , {
            "model": "gpt-4o",       # 3
            "api_key": os.environ["OPENAI_API_KEY"],
        }
    ]
}

In [7]:
manager_pa = UserProxyAgent(
    name="Manager",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config=False,
    llm_config=llm_config["config_list"][1],
    system_message="""
    You are a marketing campain manager. 
    your role is to provide campain topics and evaluate the ideas and scripts
    generated by the createve team.
    """
)


In [6]:
idea_gen = ConversableAgent(
    name="Idea_Generator",
    human_input_mode="NEVER",
    llm_config=llm_config["config_list"][0],
    system_message="""
    You are a marketing campain idea creator.
    Your role is to
    1. Generate innovate campain ideas based on the given topic.
    2. Suggest potential storylines and characters for video content
    3. Provide a brief outline for each idea
    Present 3 to 5 unique ideas for each request
    """
)

In [8]:
script_writer = ConversableAgent(
    name="Script_Writer",
    human_input_mode="NEVER",
    llm_config=llm_config["config_list"][0],
    system_message="""
    You are a marketing campain script writer.
    Your role is to
    1. Generate scripts for each idea
    2. Provide a brief outline for each script
    3. Present 3 to 5 unique scripts for each request
    """
)

In [9]:
script_reviewer = ConversableAgent(
    name="Script_Reviewer",
    human_input_mode="NEVER",
    llm_config=llm_config["config_list"][0],
    system_message="""
    You are a marketing campain script reviewer.
    Your role is to
    1. Review the scripts and provide feedback
    2. Provide a brief outline for each review
    3. Present 3 to 5 unique reviews for each request
    """
)

In [18]:
chats = [
    {
        "sender": manager_pa,
        "recipient": idea_gen,
        "message": 
            "Hello, I need creative ideas for our new marketing campaign. "
            "Our product is eco-friendly home cleaning products. "
            "Could you generate 3-5 innovative campaign ideas, including potential "
            "storylines and characters for video content?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Summarize the generated ideas "
                             "into a JSON object: "
                             "{'idea1': {'concept': '', 'storyline': '', 'characters': ''}, "
                             "'idea2': {...}, 'idea3': {...}}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": manager_pa,
        "recipient": script_writer,
        "message": 
            "Great ideas! Let's develop a script for the following concept: "
            "[INSERT SELECTED IDEA HERE]. "
            "Please write a full script including dialogue, scene descriptions, "
            "and camera directions. Aim for a 2-3 minute video.",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Provide a brief summary of the script "
                             "as a JSON object: "
                             "{'title': '', 'main_characters': [], 'key_scenes': [], "
                             "'estimated_duration': ''}",
        },
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": manager_pa,
        "recipient": script_reviewer,
        "message": "Please review the following script: [INSERT SCRIPT HERE]. "
                   "Analyze it for effectiveness, engagement, and alignment with "
                   "our eco-friendly cleaning products campaign. Provide constructive "
                   "feedback and suggest improvements.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Summarize the review points "
                             "as a JSON object: "
                             "{'strengths': [], 'weaknesses': [], 'suggestions': []}",
        },
    },
    {
        "sender": manager_pa,
        "recipient": manager_pa,
        "message": "Based on the script and review, decide if the script is ready "
                   "for production or needs revisions. If revisions are needed, "
                   "specify the key areas to address.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Summarize the decision "
                             "as a JSON object: "
                             "{'decision': 'ready'/'needs_revision', "
                             "'key_revision_areas': []}",
        },
    },
]

In [19]:
from autogen import initiate_chats
chat_results = initiate_chats( chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Manager (to Idea_Generator):

Hello, I need creative ideas for our new marketing campaign. Our product is eco-friendly home cleaning products. Could you generate 3-5 innovative campaign ideas, including potential storylines and characters for video content?

--------------------------------------------------------------------------------
Idea_Generator (to Manager):

Certainly! Here are five innovative marketing campaign ideas for your eco-friendly home cleaning products, along with potential storylines and characters for video content:

1. **"The Green Home Makeover Challenge"**
   - **Concept:** Invite families or influencers to transform their homes using your eco-friendly products.
   - **Storyline & Characters:** Follow a charismatic home renovation expert and a family eager to adopt sustainable 

In [20]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "idea1": {
    "concept": "The Green Home Makeover Challenge",
    "storyline": "Families or influencers transform their homes using eco-friendly products, showcasing safe, effective cleaning and sustainable living tips.",
    "characters": "A home renovation expert and participating families."
  },
  "idea2": {
    "concept": "Eco-Heroes: The Clean Squad",
    "storyline": "Animated superhero team battles household dirt and pollution, using eco-friendly products to restore cleanliness and educate viewers on environmental benefits.",
    "characters": "Captain Green, Miss Biodegradable, Spray Boy, and other eco-hero characters."
  },
  "idea3": {
    "concept": "A Day in the Life of a Sustainable Home",
    "storyline": "Follow diverse households integrating eco-friendly cleaning into their daily routines, highlighting benefits and inspiring viewers to adopt sustainable habits.",
    "characters": "Real families or individuals committed to eco-conscious living."
  },
  "idea4": {
 

In [21]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.0015064000000000002, 'gpt-4.1-mini-2025-04-14': {'cost': 0.0008076000000000001, 'prompt_tokens': 707, 'completion_tokens': 328, 'total_tokens': 1035}, 'gpt-4.1-nano-2025-04-14': {'cost': 0.0006988000000000001, 'prompt_tokens': 2448, 'completion_tokens': 1135, 'total_tokens': 3583}}, 'usage_excluding_cached_inference': {'total_cost': 0.0015064000000000002, 'gpt-4.1-mini-2025-04-14': {'cost': 0.0008076000000000001, 'prompt_tokens': 707, 'completion_tokens': 328, 'total_tokens': 1035}, 'gpt-4.1-nano-2025-04-14': {'cost': 0.0006988000000000001, 'prompt_tokens': 2448, 'completion_tokens': 1135, 'total_tokens': 3583}}}


{'usage_including_cached_inference': {'total_cost': 0.0009677000000000002, 'gpt-4.1-mini-2025-04-14': {'cost': 0.0008076000000000001, 'prompt_tokens': 707, 'completion_tokens': 328, 'total_tokens': 1035}, 'gpt-4.1-nano-2025-04-14': {'cost': 0.0001601, 'prompt_tokens': 973, 'completion_tokens': 157, 'total_tokens': 1130}},